In [1]:
import os
import numpy as np
import cv2 as cv
import pickle
from sklearn.cluster import KMeans
import pandas as pd
import sys

In [2]:
datasetPath = "coco128\\images\\train2017\\"
data = os.listdir(datasetPath) 

In [3]:
len(data)

352

In [4]:
def makeDesc(imagePath):
    sift = cv.SIFT_create()
    
    img = cv.imread(imagePath)
    imgGray= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
    point, desc = sift.detectAndCompute(imgGray,None)
    
    return desc

In [5]:
descs = np.empty([0, 128])

for nameFile in data:
    totalImagePath = datasetPath + nameFile
    descs = np.concatenate((descs,makeDesc(totalImagePath)))

In [6]:
descs.shape

(378452, 128)

In [65]:
k_means = KMeans(n_clusters=1024)
k_means.fit(descs)

KMeans(n_clusters=1024)

In [7]:
sift = cv.SIFT_create()

def makeHist(imagePath):
    img = cv.imread(imagePath)
    
    if(img is not None):
        grayImg= cv.cvtColor(img,cv.COLOR_BGR2GRAY)
        pkeys, descs = sift.detectAndCompute(grayImg,None)
        
        if len(pkeys) == 0:
            return np.array([])
        
        kmeansPredict = k_means.predict(descs.astype(float))
        hist = np.histogram(kmeansPredict, bins=1024)[0]
        return hist / np.linalg.norm(hist)
    
    else:
        return []

In [8]:
datasetPath2 = "VOCdevkit\\VOC2012\\JPEGImages"
data2 = os.listdir(datasetPath2) 

In [ ]:
idImg = 0
totalData = []

for nameFile in data2: 
    totalImagePath = datasetPath2 + "\\" + nameFile
    hist = makeHist(totalImagePath)
    
    if len(vec) == 0:
        print(path)
        continue
        
    row = {"id": idImg, "path":totalImagePath, "vector":hist}
    idImg+=1
    
    totalData.append(row)

In [38]:
df = pd.DataFrame(totalData)

In [39]:
np.set_printoptions(threshold=sys.maxsize)
df.to_csv("data.csv", index=False, float_format='%:.17f', sep=',')

In [40]:
with open("k_means.pkl", "wb") as f:
    pickle.dump(k_means, f)